# *In silico* Multi-Objective BO with Odyssey

We are now moving onto the Multi-Objective Bayesian Optimization (MOBO)! Just like our previous exercise in `05_Bayesian_Optimization_Odyssey_InSilico.ipynb`, we will continue to use [**Odyssey**](https://odyssey-51edb0.gitlab.io/). The only difference is that we will now expand on the single-objective opmitization to muti-objective optimization. As usual, you will move on to the PumpBot in the next exercise, so make sure you do proper coding and experiment with parameters in this exercise to make the next one easier. 

Let's start by importing `Odyssey` and some additional packages.

In [ ]:
# Odyssey 
from odyssey.mission import Mission # Mission
from odyssey.objective import Objective # Objective
from odyssey.navigators import qNParEGO_Navigator, qEHVI_Navigator # Multi-Objective Navigators
from odyssey.navigators.sampler_navigators import Sobol_Navigator, Grid_Navigator # Sampler

# Other Packages
import torch
import numpy as np
import pandas as pd   
import matplotlib.pyplot as plt
from IPython import display
from warnings import catch_warnings, simplefilter

from botorch.utils.multi_objective.pareto import is_non_dominated
from botorch.utils.multi_objective.box_decompositions.non_dominated import NondominatedPartitioning


The key difference in the imported classes are the multi-objective naviators (acquisition functions): `qNParEGO_Navigator` and `qEHVI_Navigator`. These respectively refers **Par**eto **E**fficient **G**lobal **O**ptimization (ParEGO) and **E**xpected **H**ypervolume **I**mprovement (EHVI). 

Note that the prefix `q` in the navigator name represents the batch size, indicating the number of points to be evaluated simultaneously in a parallel fashion. The prefix `N` stands for "noisy", referring to the handling of noise in the objective function of constraints. Therefore, `qNParEGO` is a parallel, noisy version of the `ParEGO` algorithm, for example. We will not be dealing with the batch size, so you do not need to worry about it.

## Exercise 7.1: Multi-Objective Optimization using single input function

As usual, we start with a simple example. This case, we provide you with the objective functions, and you will be trying to find the global minimum of two functions below for $x \in [-5, 5]$.

In [ ]:
def first_objective_function(x):
    val = -(torch.sin(x) + torch.sin((10.0 / 3.0) * x))
    return val

In [ ]:
def second_objective_function(x):
    val = -(torch.cos((2.0 / 1.0) * x) + torch.cos((10.0 / 3.0) * x))
    return val

Let's visualize the two objective functions to see where the minimum values of the two functions are.

In [ ]:
test_X = torch.linspace(-5, 5, 1000)
test_Y1 = first_objective_function(test_X)
test_Y2 = second_objective_function(test_X)


plt.figure(figsize=(7, 5))
plt.plot(test_X, test_Y1, color='black', label=r"$f_1(x)$")
plt.plot(test_X, test_Y2, color='red', label=r"$f_2(x)$")
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.show()

Not surprisingly, you can see that the global minima of the two objective functions do not coincide, meaning that you cannot simultaneously minimize both $f_1(x)$ and $f_2(x)$. It is time to apply what you learned about the multi-objective optimization. 

Let's use MOBO to find the optimal trade-offs between the two conflicting objectives! We start by defining the two objectives below. 

In [ ]:
objectives = [Objective(func=first_objective_function),
              Objective(func=second_objective_function)]

We now initialize our "mission" where we perform the minimization of $f_1(x)$ and $f_2(x)$ for $x \in [-5, 5]$. Note that `func` and `maneuvers` are list of length two since we optimize two objectives. 

In [ ]:
param_space = [(-5.0, 5.0)]

mission = Mission(name="MOO minimizing two functions",
                  funcs=objectives,
                  maneuvers=['descend', 'descend'],
                  envelope=param_space)

The next step is to specify how we would like to navigate through our search space by selecting the `Navigator`. We demonstrate how to use `qNParEGO_Navigator` that scalarizaes the multi-objective into a single-objective problem. 

<div class="alert alert-block alert-info">
<b>Note:</b> Aftr you run this simple tutorial, change to 'qEHVI_Navigator' and re-run the example and compare its performance/behavior.
</div>

In [ ]:
num_init = 5
navigator = qNParEGO_Navigator(mission=mission,
                               num_init_design=num_init,
                               input_scaling=False,
                               data_standardization=False,
                               init_method=Sobol_Navigator(mission=mission),
                               acq_function_params={}
)

The function below is used to plot Pareto Front.

In [ ]:
def plot_progress():    
    plt.figure(figsize=(10, 6))
    pareto = mission.display_Y[is_non_dominated(Y=mission.display_Y, maximize=False)]
    pareto_sorted = pareto.cpu().numpy()
    pareto_front = pareto_sorted[pareto_sorted[:, 0].argsort()]

    plt.plot(pareto_front[:, 0], pareto_front[:, 1], label='qNParEGO Pareto Front', marker='+', markersize=10, color='royalblue', linestyle='-')
    plt.scatter(mission.display_Y[:, 0].cpu(), mission.display_Y[:, 1].cpu(), label='qNParEGO Samples', alpha=0.5)

    plt.xlabel(r'$f_1(x)$')
    plt.ylabel(r'$f_2(x)$')
    plt.title('Pareto Front using qNParEGO')
    plt.legend()
    plt.show()

Now, let's start the MOBO loop!

You are **strongly** encouraged to experiment with the parameters.

In [ ]:
num_iter = 15
opt_samples = len(mission.display_X) - num_init
ref_point = torch.tensor([5.0, 5.0])


while opt_samples < num_iter:
    with catch_warnings() as w:
        simplefilter('ignore')
        trajectory = navigator.trajectory()
        observation = navigator.probe(input_data=trajectory, init=False)
        navigator.relay(trajectory, observation)
        navigator.upgrade()

    # Compute Pareto front and hypervolume
    pareto_mask = is_non_dominated(Y=mission.display_Y, maximize=False)
    pareto_Y = mission.display_Y[pareto_mask]

    # Update training data
    partitioning = NondominatedPartitioning(ref_point=ref_point, Y=pareto_Y)
    hypervolume = partitioning.compute_hypervolume().item()

    opt_samples = len(mission.display_X) - num_init
    
    display.clear_output(wait=True) # This will help you remove figures as you go
    plot_progress()
    print(f"Iteration {opt_samples}: Hypervolume = {hypervolume:.4f}")
    input('Press Enter to proceed.')
    
print(f"Finished optimization.")


## Exercise 7.2: Let's combine color mixing with different pH values

Now you got a good taste of how to perform MOBO using Odyssey. It is time to get back to the color mixing. 

We will build upon the previous notebooks to perform inSilico color mixing, so we only introduce you to the "new" features that are relevant for MOBO. In this case, we will utilize the functionality of the `SilicoPumpController` to assign a pH value for each of the red, green, blue, and yellow solution. It can calculate the pH of the solution mixtures, in addition to the resulting color. 

Reusing your code from the previous *in silico* exercises as much as you can to figure out how to perform MOBO. 
 

In [ ]:
# Pump Controller
from pump_controller import SilicoPumpController

In [ ]:
# Let's have some noise to keep it real.
silicobot = SilicoPumpController(noise_std = 3)

We can introduce the pH value of each of the stock solution using `set_ph_coefficient` method. Let's assume the following pH values:
* red: 7.0
* green: 10.0
* blue: 4.0
* yellow: 3.0

In [ ]:
silicobot.set_ph_coefficient(r=7.0, g=10.0, b=4.0, y=3.0)

Now, let's test how it behaves. Let's mix half red and half green to make yellow. Since the pH value of red and green are 7.0 and 10.0, respectively, the resulting solution should have the pH level of 8.5.

In [ ]:
silicobot.mix_color(col_list=[0.5, 0.5, 0.0, 0.0], ph=True)

You can see that the resulting RGB corresponds to the yellow color (equal parts of red and green), and you get the pH value of 8.5. Of course, with some errors due to the `noise_std` value. 

Here are 3 different sets of colors and pH values. Try to obtain them using MOBO feature of Odyssey. Be sure to tinker with the notebook to see how you can extract the key information such as the ratios of `r`, `g`, `b`, and `y` to obtain the "best" mixtures.

Have fun!

In [ ]:
target_color_1 = [125., 125., 0.]
target_pH_1 = 3.

In [ ]:
target_color_2 = [83., 156., 72.]
target_pH_2 = 9.

In [ ]:
target_color_3 = [145., 120., 100.]
target_pH_3 = 7.

In [ ]:
# place for your code

In [ ]:
# place for your code

In [ ]:
# place for your code.  Add more as needed.